In [2]:
import pandas as pd
import os
exp_name = 'combined'
file_name = 'combined_noise_results.csv'
path = f"../csv_results/{exp_name}/{file_name}"
df = pd.read_csv(path)

In [3]:
df.describe()

,SNR,EER
count,120.000000,120.000000
mean,10.000000,0.043797
std,7.100716,0.036351
min,0.000000,0.010278
25%,5.000000,0.021292
50%,10.000000,0.032648
75%,15.000000,0.050761
max,20.000000,0.209098


In [4]:
df.head(30)

,Noise,SNR,Model,EER
0,noisy_gaussian,0,campplus,0.079633
1,noisy_gaussian,0,ecapa_tdnn_ft,0.085559
2,noisy_gaussian,0,ecapa_tdnn,0.081721
3,noisy_gaussian,0,redimnet,0.074895
4,noisy_gaussian,10,campplus,0.033785
5,noisy_gaussian,10,ecapa_tdnn_ft,0.042374
6,noisy_gaussian,10,ecapa_tdnn,0.036264
7,noisy_gaussian,10,redimnet,0.029624
8,noisy_gaussian,15,campplus,0.024607
9,noisy_gaussian,15,ecapa_tdnn_ft,0.033777


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Noise   120 non-null    object 
 1   SNR     120 non-null    int64  
 2   Model   120 non-null    object 
 3   EER     120 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.9+ KB


# Only base models
- delete ecapa_tdnn_ft from dataframe

In [6]:
df = df[df['Model'] != 'ecapa_tdnn_ft']

In [7]:
# Use existing DataFrame 'df' and matplotlib which is already imported
from matplotlib import pyplot as plt


# Define model name mapping
model_mapping = {
    'ecapa_tdnn': 'ECAPA',
    'campplus': 'CAM++',
    'redimnet': 'REDIMNET'
}

model_color_mapping = {
    'ecapa_tdnn':  'blue',
    'campplus': 'orange',
    'redimnet': 'green'
}
noise_types = df['Noise'].unique()
noise_mapping = {
    'AirConditioner': 'Air Conditioner',
    'Babble': 'Babble',
    'Neighbor': 'Neighbour',
    'noisy_gaussian': 'Gaussian',
    'noisy_poisson': 'Poisson',
    'noisy_rayleigh': 'Rayleigh',
}

max_ylim = 33

In [8]:
# Plot for each noise type
for noise in noise_types:
    plt.figure(figsize=(4, 6), dpi=300)

    # Plot for each model
    for model in model_mapping.keys():
        # Filter data for current noise and model
        mask = (df['Noise'] == noise) & (df['Model'] == model)
        data = df[mask].sort_values('SNR')

        plt.plot(
            data['SNR'],
            data['EER'] * 100,  # Convert to percentage
            marker="o",
            label=model_mapping[model],
            linestyle="-",
            alpha=0.6,
            color=model_color_mapping[model],  
        )

    plt.title(f"EER vs SNR for {noise_mapping[noise]} Noise")
    plt.xlabel("SNR [dB]")
    plt.ylabel("EER [%]")
    plt.ylim(0, max_ylim)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    # plt.show()  # Uncommenting to display the plot

    # save the figure into ../figures/
    figure_path = (
        f"../figures/{exp_name}/{exp_name}_{noise_mapping[noise]}_eer_vs_snr.png"
    )
    os.makedirs(os.path.dirname(figure_path), exist_ok=True)
    plt.savefig(figure_path, dpi=300)
    plt.close()

# Only compare ecapa_tdnn_ft with ecapa_tdnn
-load df again

In [10]:
# load the data again
df = pd.read_csv(path)
# only keep the models we want to plot ecapa_tdnn, ecapa_tdnn_ft
df = df[df['Model'].isin(['ecapa_tdnn', 'ecapa_tdnn_ft'])]
df.head(30)

,Noise,SNR,Model,EER
1,noisy_gaussian,0,ecapa_tdnn_ft,0.085559
2,noisy_gaussian,0,ecapa_tdnn,0.081721
5,noisy_gaussian,10,ecapa_tdnn_ft,0.042374
6,noisy_gaussian,10,ecapa_tdnn,0.036264
9,noisy_gaussian,15,ecapa_tdnn_ft,0.033777
10,noisy_gaussian,15,ecapa_tdnn,0.025958
13,noisy_gaussian,20,ecapa_tdnn_ft,0.028489
14,noisy_gaussian,20,ecapa_tdnn,0.020257
17,noisy_gaussian,5,ecapa_tdnn_ft,0.057896
18,noisy_gaussian,5,ecapa_tdnn,0.052242


In [ ]:
# Use existing DataFrame 'df' and matplotlib which is already imported
from matplotlib import pyplot as plt


# Define model name mapping
model_mapping = {"ecapa_tdnn": "ECAPA", "ecapa_tdnn_ft": "ECAPA FT"}

model_color_mapping = {"ecapa_tdnn": "blue", "ecapa_tdnn_ft": "red"}
noise_types = df["Noise"].unique()
noise_mapping = {
    "AirConditioner": "Air Conditioner",
    "Babble": "Babble",
    "Neighbor": "Neighbour",
    "noisy_gaussian": "Gaussian",
    "noisy_poisson": "Poisson",
    "noisy_rayleigh": "Rayleigh",
}

max_ylim = 25

In [13]:
# Plot for each noise type
for noise in noise_types:
    plt.figure(figsize=(4, 6), dpi=300)

    # Plot for each model
    for model in model_mapping.keys():
        # Filter data for current noise and model
        mask = (df["Noise"] == noise) & (df["Model"] == model)
        data = df[mask].sort_values("SNR")

        plt.plot(
            data["SNR"],
            data["EER"] * 100,  # Convert to percentage
            marker="o",
            label=model_mapping[model],
            linestyle="-",
            alpha=0.6,
            color=model_color_mapping[model],
        )

    plt.title(f"EER vs SNR for {noise_mapping[noise]} Noise")
    plt.xlabel("SNR [dB]")
    plt.ylabel("EER [%]")
    plt.ylim(0, max_ylim)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    # plt.show()  # Uncommenting to display the plot

    # save the figure into ../figures/
    figure_path = (
        f"../figures/finetune/{exp_name}_{noise_mapping[noise]}_eer_vs_snr.png"
    )
    os.makedirs(os.path.dirname(figure_path), exist_ok=True)
    plt.savefig(figure_path, dpi=300)
    plt.close()

In [15]:
# Set figure size
plt.figure(figsize=(8, 6), dpi=300)

sorted_noise_types = sorted(noise_types, key=lambda x: x)

# Create bar plot with error bars showing quartiles
for i, model in enumerate(model_mapping.keys()):
    model_data = [
        df[(df["Model"] == model) & (df["Noise"] == dist)]["EER"] * 100
        for dist in sorted_noise_types
    ]
    # Plot with slight offset for each model
    positions = [x + i * 0.25 for x in range(len(sorted_noise_types))]
    bp = plt.boxplot(
        model_data,
        positions=positions,
        labels=[""] * len(sorted_noise_types),
        widths=0.2,
        patch_artist=True,
        boxprops=dict(facecolor=model_color_mapping[model], alpha=0.5),
        medianprops=dict(color="black"),
    )
    # Add to legend
    plt.plot([], [], color=model_color_mapping[model], label=model_mapping[model])

# Set x-axis labels using noise_mapping
plt.xticks(
    [x + 0.25 for x in range(len(sorted_noise_types))],
    [noise_mapping[noise] for noise in sorted_noise_types],
)

plt.xlabel("Noise Types")
plt.ylabel("EER [%]")
plt.title("Results by Model and Noise Type")
plt.ylim(0)
plt.legend()
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()

# Save the figure into ../figures/
figure_path = f"../figures/finetune/finetune_boxplot.png"
os.makedirs(os.path.dirname(figure_path), exist_ok=True)
plt.savefig(figure_path, dpi=300)
plt.close()

C:\Users\filip\AppData\Local\Temp\ipykernel_1596\566797394.py:14: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = plt.boxplot(
C:\Users\filip\AppData\Local\Temp\ipykernel_1596\566797394.py:14: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = plt.boxplot(


In [17]:
# save df to csv with finetune resulst 
path = f"../csv_results/finetune/finetune_combined_noise_results.csv"
os.makedirs(os.path.dirname(path), exist_ok=True)
df.to_csv(
    path, index=False
)


# Summary

In [ ]:
# Set figure size
plt.figure(figsize=(8, 6), dpi=300)

sorted_noise_types = sorted(noise_types, key=lambda x: x)

# Create bar plot with error bars showing quartiles
for i, model in enumerate(model_mapping.keys()):
    model_data = [
        df[(df["Model"] == model) & (df["Noise"] == dist)]["EER"] * 100
        for dist in sorted_noise_types
    ]
    # Plot with slight offset for each model
    positions = [x + i * 0.25 for x in range(len(sorted_noise_types))]
    bp = plt.boxplot(
        model_data,
        positions=positions,
        labels=[""] * len(sorted_noise_types),
        widths=0.2,
        patch_artist=True,
        boxprops=dict(facecolor=model_color_mapping[model], alpha=0.5),
    )
    # Add to legend
    plt.plot([], [], color=model_color_mapping[model], label=model_mapping[model])

# Set x-axis labels using noise_mapping
plt.xticks(
    [x + 0.25 for x in range(len(sorted_noise_types))], 
    [noise_mapping[noise] for noise in sorted_noise_types]
)

plt.xlabel('Noise Types')
plt.ylabel('EER [%]')
plt.title('Results by Model and Noise Type')
plt.ylim(0)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

# Save the figure into ../figures/
figure_path = f"../figures/{exp_name}/{exp_name}_boxplot.png"
os.makedirs(os.path.dirname(figure_path), exist_ok=True)
plt.savefig(figure_path, dpi=300)
plt.close()

/var/folders/nc/8gf92v353514742zf9ytrs600000gn/T/ipykernel_16441/351460442.py:14: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = plt.boxplot(
/var/folders/nc/8gf92v353514742zf9ytrs600000gn/T/ipykernel_16441/351460442.py:14: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = plt.boxplot(
/var/folders/nc/8gf92v353514742zf9ytrs600000gn/T/ipykernel_16441/351460442.py:14: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = plt.boxplot(


# Noisy and Noisy Bg

In [ ]:
import pandas as pd
import os

exp_name = "noisy"
path = f"../csv_results/{exp_name}/{exp_name}_results.csv"
df = pd.read_csv(path)